In [1]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Dataset, Subset
from torchvision import transforms, models
import pytorch_lightning as pl
from pytorch_lightning.utilities.model_summary import ModelSummary
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
import wandb
import torchmetrics

In [2]:
wandb.login()
wandb.init(
      mode='disabled',
      # Set the project where this run will be logged
      project="histopathologic-cancer-classifier", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name="Test4", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.005,
      "data_size": 0.001,
      "batch_size":4,
      })

wandb: Currently logged in as: respwill. Use `wandb login --relogin` to force relogin


In [3]:
# backbone = models.resnet50(weights='DEFAULT')
# num_filters = backbone.fc.in_features
# layers = list(backbone.children())[:-1]
# self.feature_extractor = nn.Sequential(*layers)
# self.classifier = nn.Linear(num_filters, num_target_classes)

In [4]:
class DataModule(pl.LightningDataModule):
    def __init__(self, batch_size, train, val):
        super().__init__()
        self.batch_size = batch_size
        self.dims = (3, 96, 96)
        self.num_classes = 1
        self.train = train
        self.val = val
    
    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size)
        
        

In [5]:
class ImageEncoder(pl.LightningModule):
    def __init__(self, input_shape, num_classes, learning_rate=2e-4):
        super().__init__()
        
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=10, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=1),
            nn.ReLU()
        )
        n_sizes = self._get_conv_output((3,96,96))
        
        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(n_sizes, 320),
            nn.ReLU(),

            nn.Linear(320, 50),
            nn.ReLU(),

            nn.Linear(50, 1)

        )
        
        self.accuracy = torchmetrics.Accuracy(task='binary')
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.dense(x)
        return torch.sigmoid(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.binary_cross_entropy(y_hat, y)
        pred = torch.argmax(y_hat, dim=1)
        return {"loss":loss, 'pred':pred, 'y':y}
    
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        pred = torch.stack([x['pred'] for x in outputs]).to(device=torch.device('cuda'))
        y = torch.stack([x['y'] for x in outputs]).to(torch.int32)
        y = torch.squeeze(y)
        avg_acc = self.accuracy(pred, y)
        self.log("train_loss", avg_loss, prog_bar=True)
        self.log("train_acc", avg_acc, prog_bar=True)
        wandb.log({'train_loss': avg_loss, 'learning_rate': self.lr_schedulers().get_last_lr()[0], 'train_acc':avg_acc})
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        pred = torch.argmax(y_hat, dim=1)
        val_loss = F.binary_cross_entropy(y_hat, y)
        return {'val_loss':val_loss,'pred':pred, 'y':y}
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        pred = torch.stack([x['pred'] for x in outputs]).to(device=torch.device('cuda')).view(1,-1)
        y = torch.stack([x['y'] for x in outputs]).to(torch.int32)
        
        print('pred:', pred)
        print('y:', y)
        avg_acc = self.accuracy(pred, y)
        self.log("val_loss", avg_loss, prog_bar=True)
        self.log("val_acc", avg_acc, prog_bar=True)
        wandb.log({'val_loss': avg_loss, 'val_acc':avg_acc})
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = CosineAnnealingLR(optimizer, T_max=100, eta_min=0.00001)
        return [optimizer], [scheduler]
    
    def _get_conv_output(self, shape):
        batch_size = 1
        input = torch.autograd.Variable(torch.rand(batch_size, *shape))
        output_feat = self.encoder(input) 
        n_size = output_feat.data.view(batch_size, -1).size(1)
        n_size = n_size
        return n_size
    


In [6]:
class CancerDataset(Dataset):
    def __init__(self, label_dir, img_dir):
        self.img_labels = pd.read_csv(label_dir)
        self.img_dir = img_dir
        self.transform = transforms.ToTensor()
    
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        # label file has two columns: id, label
        # to read each images from dir
        img_path = self.img_dir + '/'+ self.img_labels.iloc[idx,0] + '.tif'
        # read image as numpy array and normalize it
        image = Image.open(img_path)
        image_array = self.transform(image) / 255.0
        
        # read label
        label = self.img_labels.iloc[idx, 1].astype('float32')
        label = torch.Tensor([label])
        # return image array and label
        return image_array, label

In [7]:
data = CancerDataset(label_dir=r'../histopathologic-cancer-detection/histopathologic-cancer-detection_data/train_labels.csv',
                          img_dir=r'../histopathologic-cancer-detection/histopathologic-cancer-detection_data/train/')

In [8]:
train_n = int(len(data) * wandb.config['data_size'] * 0.7)
val_n = int(len(data) * wandb.config['data_size'] * 0.3)

In [9]:
train_n, val_n

(154, 66)

In [10]:
def train_val_dataset(dataset, train_split=0.75, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), train_size=train_split, test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

In [11]:
dataset = train_val_dataset(data, train_split=train_n, val_split=val_n)

In [12]:
# train_dataloader = DataLoader(dataset['train'], batch_size=wandb.config['batch_size'])
# val_dataloader = DataLoader(dataset['val'], batch_size=wandb.config['batch_size'])

In [13]:
dm = DataModule(batch_size=wandb.config['batch_size'], train=dataset['train'], val=dataset['val'])

C:\Users\jaydenpark\Anaconda3\envs\nlp3\lib\site-packages\pytorch_lightning\core\datamodule.py:149: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


In [14]:
im_encoder = ImageEncoder(input_shape=dm.size(), num_classes=dm.num_classes, learning_rate=wandb.config['learning_rate'])

C:\Users\jaydenpark\Anaconda3\envs\nlp3\lib\site-packages\pytorch_lightning\core\datamodule.py:158: LightningDeprecationWarning: DataModule property `size` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `size` was deprecated in v1.5 and will be removed in v1.7.")
C:\Users\jaydenpark\Anaconda3\envs\nlp3\lib\site-packages\pytorch_lightning\core\datamodule.py:144: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


In [15]:
ModelSummary(im_encoder, max_depth=-1)

   | Name      | Type       | Params
------------------------------------------
0  | encoder   | Sequential | 5.8 K 
1  | encoder.0 | Conv2d     | 760   
2  | encoder.1 | ReLU       | 0     
3  | encoder.2 | Conv2d     | 5.0 K 
4  | encoder.3 | ReLU       | 0     
5  | encoder.4 | MaxPool2d  | 0     
6  | encoder.5 | ReLU       | 0     
7  | dense     | Sequential | 47.4 M
8  | dense.0   | Flatten    | 0     
9  | dense.1   | Linear     | 47.3 M
10 | dense.2   | ReLU       | 0     
11 | dense.3   | Linear     | 16.1 K
12 | dense.4   | ReLU       | 0     
13 | dense.5   | Linear     | 51    
14 | accuracy  | Accuracy   | 0     
------------------------------------------
47.4 M    Trainable params
0         Non-trainable params
47.4 M    Total params
189.426   Total estimated model params size (MB)

In [16]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath='./check_point/', filename='{epoch}-{train_loss:.4f}-{val_loss:.4f}', 
    monitor="val_loss", mode="min", save_top_k=5
)

In [17]:
trainer = pl.Trainer(accelerator='gpu',
                     devices=1,
                    max_epochs=200,
                    callbacks=[checkpoint_callback])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [18]:
trainer.fit(im_encoder, dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name     | Type       | Params
----------------------------------------
0 | encoder  | Sequential | 5.8 K 
1 | dense    | Sequential | 47.4 M
2 | accuracy | Accuracy   | 0     
----------------------------------------
47.4 M    Trainable params
0         Non-trainable params
47.4 M    Total params
189.426   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\jaydenpark\Anaconda3\envs\nlp3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


pred: tensor([[0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')
y: tensor([[[0],
         [0],
         [0],
         [1]],

        [[1],
         [0],
         [0],
         [0]]], device='cuda:0', dtype=torch.int32)


ValueError: The `preds` and `target` should have the same first dimension.